In [15]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("2_SchemaDetails")
    .master("local[*]")
    .getOrCreate()
)

spark


In [16]:
# Emp Data & Schema
from pyspark.sql.types import StructType, StructField, StringType

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

# emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"


emp_schema = StructType([
    StructField("employee_id", StringType(), True),
    StructField("department_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("age", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", StringType(), True),
    StructField("hire_date", StringType(), True)
])


In [17]:
#create a data frame

df = spark.createDataFrame(data = emp_data, schema = emp_schema)

In [18]:
# show schema
df.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [7]:
# show schema in spark native notation.

df.schema

StructType([StructField('employee_id', StringType(), True), StructField('department_id', StringType(), True), StructField('name', StringType(), True), StructField('age', StringType(), True), StructField('gender', StringType(), True), StructField('salary', StringType(), True), StructField('hire_date', StringType(), True)])

In [19]:
df.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [25]:
# Casting Column
# select employee_id, name, age, cast(salary as double) as salary from emp

from pyspark.sql.functions import expr

df.select(expr("cast(salary as double) as salary")).printSchema()

df.selectExpr("cast(salary as double)").printSchema()

df.select(col("salary").cast("double")).printSchema()


root
 |-- salary: double (nullable = true)

root
 |-- salary: double (nullable = true)

root
 |-- salary: double (nullable = true)



In [36]:
df1 = df.select("name", "age", expr("cast(salary as double)"))

In [41]:
df1.printSchema()
df1.show(5)

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: double (nullable = true)

+----------+---+-------+
|      name|age| salary|
+----------+---+-------+
|  John Doe| 30|50000.0|
|Jane Smith| 25|45000.0|
| Bob Brown| 35|55000.0|
| Alice Lee| 28|48000.0|
| Jack Chan| 40|60000.0|
+----------+---+-------+
only showing top 5 rows



In [44]:
# Adding Columns
# select employee_id, name, age, salary, (salary * 0.2) as tax from emp_casted

df2 = df1.withColumn("tax", col("salary") * 0.2 )

df2.printSchema()

df2.show(5)

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- tax: double (nullable = true)

+----------+---+-------+-------+
|      name|age| salary|    tax|
+----------+---+-------+-------+
|  John Doe| 30|50000.0|10000.0|
|Jane Smith| 25|45000.0| 9000.0|
| Bob Brown| 35|55000.0|11000.0|
| Alice Lee| 28|48000.0| 9600.0|
| Jack Chan| 40|60000.0|12000.0|
+----------+---+-------+-------+
only showing top 5 rows



In [46]:
# Literals
# select employee_id, name, age, salary, tax, 1 as columnOne, 'two' as columnTwo from emp_taxed

from pyspark.sql.functions import lit

df3 = df2.withColumn("Lit_Integer", lit(1)).withColumn("Lit_String", lit("two"))

df3.printSchema()

df3.show(5)


root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- tax: double (nullable = true)
 |-- Lit_Integer: integer (nullable = false)
 |-- Lit_String: string (nullable = false)

+----------+---+-------+-------+-----------+----------+
|      name|age| salary|    tax|Lit_Integer|Lit_String|
+----------+---+-------+-------+-----------+----------+
|  John Doe| 30|50000.0|10000.0|          1|       two|
|Jane Smith| 25|45000.0| 9000.0|          1|       two|
| Bob Brown| 35|55000.0|11000.0|          1|       two|
| Alice Lee| 28|48000.0| 9600.0|          1|       two|
| Jack Chan| 40|60000.0|12000.0|          1|       two|
+----------+---+-------+-------+-----------+----------+
only showing top 5 rows



In [53]:
# Renaming Columns
# select employee_id as emp_id, name, age, salary, tax, columnOne, columnTwo from emp_new_cols


df3.select(expr("Lit_String as lit_string")).show(5)

df3.selectExpr("Lit_String as lit_string").show(5)

df3.withColumnRenamed("Lit_String", "lit_string").show(5)




+----------+
|lit_string|
+----------+
|       two|
|       two|
|       two|
|       two|
|       two|
+----------+
only showing top 5 rows

+----------+
|lit_string|
+----------+
|       two|
|       two|
|       two|
|       two|
|       two|
+----------+
only showing top 5 rows

+----------+---+-------+-------+-----------+----------+
|      name|age| salary|    tax|Lit_Integer|lit_string|
+----------+---+-------+-------+-----------+----------+
|  John Doe| 30|50000.0|10000.0|          1|       two|
|Jane Smith| 25|45000.0| 9000.0|          1|       two|
| Bob Brown| 35|55000.0|11000.0|          1|       two|
| Alice Lee| 28|48000.0| 9600.0|          1|       two|
| Jack Chan| 40|60000.0|12000.0|          1|       two|
+----------+---+-------+-------+-----------+----------+
only showing top 5 rows



In [57]:
#non existent columns can be renamed and they do not throw error incase of withColumnRenamed!

df.withColumnRenamed("dummy", "timmy").show(5)

df.selectExpr("dummy", "timmy").show(5)

+-----------+-------------+----------+---+------+------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|
+-----------+-------------+----------+---+------+------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|
+-----------+-------------+----------+---+------+------+----------+
only showing top 5 rows



AnalysisException: Column 'dummy' does not exist. Did you mean one of the following? [name, age, salary, gender, hire_date, employee_id, department_id]; line 1 pos 0;
'Project ['dummy, 'timmy]
+- LogicalRDD [employee_id#50, department_id#51, name#52, age#53, gender#54, salary#55, hire_date#56], false


In [58]:
# Remove Column


df3.drop("Lit_String").show(5)

+----------+---+-------+-------+-----------+
|      name|age| salary|    tax|Lit_Integer|
+----------+---+-------+-------+-----------+
|  John Doe| 30|50000.0|10000.0|          1|
|Jane Smith| 25|45000.0| 9000.0|          1|
| Bob Brown| 35|55000.0|11000.0|          1|
| Alice Lee| 28|48000.0| 9600.0|          1|
| Jack Chan| 40|60000.0|12000.0|          1|
+----------+---+-------+-------+-----------+
only showing top 5 rows



In [60]:
#Removing non existent column does not throw an error.
df3.drop("dummy").show(5)

+----------+---+-------+-------+-----------+----------+
|      name|age| salary|    tax|Lit_Integer|Lit_String|
+----------+---+-------+-------+-----------+----------+
|  John Doe| 30|50000.0|10000.0|          1|       two|
|Jane Smith| 25|45000.0| 9000.0|          1|       two|
| Bob Brown| 35|55000.0|11000.0|          1|       two|
| Alice Lee| 28|48000.0| 9600.0|          1|       two|
| Jack Chan| 40|60000.0|12000.0|          1|       two|
+----------+---+-------+-------+-----------+----------+
only showing top 5 rows



In [68]:
# Filter data 
# select employee_id as emp_id, name, age, salary, tax, columnOne from emp_col_dropped where tax > 1000

df2.show(5)

df2.where("salary > 50000").show(5)

df2.where(col("salary") > 50000).show(5)


#unlike in select, where condition need to be completely in " "
df2.where("salary" > 50000).show(5)




+----------+---+-------+-------+
|      name|age| salary|    tax|
+----------+---+-------+-------+
|  John Doe| 30|50000.0|10000.0|
|Jane Smith| 25|45000.0| 9000.0|
| Bob Brown| 35|55000.0|11000.0|
| Alice Lee| 28|48000.0| 9600.0|
| Jack Chan| 40|60000.0|12000.0|
+----------+---+-------+-------+
only showing top 5 rows

+-------------+---+-------+-------+
|         name|age| salary|    tax|
+-------------+---+-------+-------+
|    Bob Brown| 35|55000.0|11000.0|
|    Jack Chan| 40|60000.0|12000.0|
|    Jill Wong| 32|52000.0|10400.0|
|James Johnson| 42|70000.0|14000.0|
|     Kate Kim| 29|51000.0|10200.0|
+-------------+---+-------+-------+
only showing top 5 rows

+-------------+---+-------+-------+
|         name|age| salary|    tax|
+-------------+---+-------+-------+
|    Bob Brown| 35|55000.0|11000.0|
|    Jack Chan| 40|60000.0|12000.0|
|    Jill Wong| 32|52000.0|10400.0|
|James Johnson| 42|70000.0|14000.0|
|     Kate Kim| 29|51000.0|10200.0|
+-------------+---+-------+-------+
only 

TypeError: '>' not supported between instances of 'str' and 'int'

In [69]:
# LIMIT data
# select employee_id as emp_id, name, age, salary, tax, columnOne from emp_filtered limit 5

df4 = df3.limit(3)

df4.show()


+----------+---+-------+-------+-----------+----------+
|      name|age| salary|    tax|Lit_Integer|Lit_String|
+----------+---+-------+-------+-----------+----------+
|  John Doe| 30|50000.0|10000.0|          1|       two|
|Jane Smith| 25|45000.0| 9000.0|          1|       two|
| Bob Brown| 35|55000.0|11000.0|          1|       two|
+----------+---+-------+-------+-----------+----------+



In [71]:
# Bonus TIP
# Add multiple columns

new_columns_to_be_added = {
    "double_tax": col("salary") * 0.4,
    "new_column": lit("JustAValue")
}


df4.withColumns(new_columns_to_be_added).show()


+----------+---+-------+-------+-----------+----------+----------+----------+
|      name|age| salary|    tax|Lit_Integer|Lit_String|double_tax|new_column|
+----------+---+-------+-------+-----------+----------+----------+----------+
|  John Doe| 30|50000.0|10000.0|          1|       two|   20000.0|JustAValue|
|Jane Smith| 25|45000.0| 9000.0|          1|       two|   18000.0|JustAValue|
| Bob Brown| 35|55000.0|11000.0|          1|       two|   22000.0|JustAValue|
+----------+---+-------+-------+-----------+----------+----------+----------+



In [73]:
spark.stop()